In [2]:
import os
import h5py
import cv2 # pip install opencv-python

import scipy.ndimage as sp
import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.cm as cm

from PIL import Image
from scipy.ndimage.filters import gaussian_filter

%matplotlib inline

In [3]:
os.chdir('/Users/hyunwoogu/Dropbox/2019/2019Autumn/Graduation/ResearchGrant/experiment/stimuli')

In [4]:
os.listdir()

['8.png',
 '9.png',
 '14.png',
 '28.png',
 '29.png',
 '15.png',
 '17.png',
 '16.png',
 '12.png',
 '13.png',
 '11.png',
 '10.png',
 '21.png',
 '20.png',
 '22.png',
 '23.png',
 '27.png',
 '32.png',
 '26.png',
 '18.png',
 '30.png',
 '24.png',
 '25.png',
 '31.png',
 '19.png',
 '4.png',
 '5.png',
 '7.png',
 '6.png',
 '2.png',
 '3.png',
 '1.png']


## Task structure

+ 


## Subroutines

+ Given Coord. of Center Point -> Matrix of the image part
+ 

## Generate Stimuli

In [ ]:
fig = plt.figure()

plt.imshow(img)
plt.xlim(0, 1600)
plt.ylim(1200, 0)
plt.axis('off')

ax = sns.kdeplot(Data.loc[(Data.counts==0),'x'], Data.loc[(Data.counts==0),'y'], shade = True, 
                 cmap = transparent_cmap(plt.cm.Reds, trans=.99), gridsize=100)
ax.patch.set_facecolor('white')
ax.collections[0].set_alpha(0)

plt.show()

In [ ]:
FileNames = os.listdir('21/')

w,h=Image.open('21/' + FileNames[0]).size
N=len(FileNames)

arr=np.zeros((h,w,3),np.float)

for im in FileNames:
    imarr= np.array(Image.open('21/' + im),dtype=np.float)
    arr=arr+imarr/N

arr=np.array(np.round(arr),dtype=np.uint8)
out=Image.fromarray(arr,mode="RGB")
out.save("Average.png")
# out.show()

img = cv2.imread("Average.png", 1)
img = cv2.resize(img, (1600, 1200), interpolation=cv2.INTER_CUBIC)

In [ ]:
# Change current directory
os.chdir('/Users/hyunwoo/Dropbox/Needle/DMstim/')

# Access file directory
files  = list(os.walk(os.getcwd()))[0][2] # list of files in the directory
images = {} # a dictionary to include the matrix of each image

# Image downsampling in order to avoid complexity
## Although too much downsampling may hurt (e.g. :=10)
## downsampling itself may not be a huge problem, due to the simple str of input images.

downsample_factor = 7 # Arbitrarily set. It means "7 times smaller than original". 
img_row = int(383/downsample_factor)
img_col = int(300/downsample_factor)
img_chn = 3    # of channel. RGB

# File loading
for f in files:
    if f.endswith('.png'): # loda each png files from the directory to each element of the dictionary
        img = imread(f)[:,:,0:3] # 383x300x3. Erased 4th dimension of RGB, since currently all set = 1
        images[f[4:6]] = resize(img, (img_row, img_col, img_chn), 
                                anti_aliasing=True) # reduced size
        
trialInfo = pd.read_excel("subject1.xlsx", header=None)    

# 

In [ ]:
XwinSize = 150
YwinSize = 150

coord1 = (720, 440) ## x coord (from left), y coord (from top) 

fig, ax = plt.subplots(1)

plt.imshow(img)
plt.xlim(0, 1600)
plt.ylim(1200, 0)
plt.axis('off')

Start = 0
End = 30

for i in range(Start, End+1) :
    rect = patches.Rectangle(C[i, [0,1]],XwinSize,YwinSize,linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)

plt.show()